<a href="https://colab.research.google.com/github/malinphy/os_llms_colab/blob/main/YT_Chatting_with_Flan20B_UL2_using_LangChain_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install huggingface_hub langchain transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.4/969.4 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.2 MB/s eta 0:00:00


In [3]:
!pip show langchain

Name: langchain
Version: 0.0.188
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [1]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = 

In [4]:
from langchain.llms import HuggingFaceHub

In [5]:
flan_ul2 = HuggingFaceHub(
    repo_id="google/flan-ul2", 
    model_kwargs={"temperature":0.7,
                  "max_new_tokens":256})

In [13]:
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0.1 }
)

In [59]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")

# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl")

flan_t5_xxl = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature":0.9 ,
                  "max_new_tokens":256}
)

In [60]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [61]:
memory = ConversationBufferMemory()

In [62]:
conversation = ConversationChain(
    llm=flan_t5_xxl, 
    verbose=False, 
    memory=memory
)

In [63]:
conversation.predict(input="What is the capital of Turkey? Let me inform you a litte. In Turkey there is a famous city,Istanbul. But Istanbul is not the capital.")

'Bucharest is,, the capital of Romania.'

In [64]:
conversation.predict(input="England-London, Germany- Berlin, France-Paris, USA-Washington D.C., Turkey ")

'Istanbul is a seaward pier, north of the mouth of the Bosphorus Strait on the Asian mainland, lying to the east of the city of Istanbul that lies on the Turquoise Coast.'

In [65]:
conversation.predict(input="Can you speak Turkish?")

'No'

In [66]:
conversation.predict(input="Türkçe konuşabilir misin?")

"It's wrong."

## Counting the tokens

In [67]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

In [68]:
test_input = "Now Good Morning Ms Rogers"
# tokenizer([test_input])
tokenizer.tokenize(test_input) 

['▁Now', '▁Good', '▁Morning', '▁M', 's', '▁Roger', 's']

In [69]:
conversation.memory.buffer

"Human: What is the capital of Turkey? Let me inform you a litte. In Turkey there is a famous city,Istanbul. But Istanbul is not the capital.\nAI: Bucharest is,, the capital of Romania.\nHuman: England-London, Germany- Berlin, France-Paris, USA-Washington D.C., Turkey \nAI: Istanbul is a seaward pier, north of the mouth of the Bosphorus Strait on the Asian mainland, lying to the east of the city of Istanbul that lies on the Turquoise Coast.\nHuman: Can you speak Turkish?\nAI: No\nHuman: Türkçe konuşabilir misin?\nAI: It's wrong."

In [70]:
formatted_prompt = conversation.prompt.format(input='the next input',history=memory.buffer)
formatted_prompt

"The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: What is the capital of Turkey? Let me inform you a litte. In Turkey there is a famous city,Istanbul. But Istanbul is not the capital.\nAI: Bucharest is,, the capital of Romania.\nHuman: England-London, Germany- Berlin, France-Paris, USA-Washington D.C., Turkey \nAI: Istanbul is a seaward pier, north of the mouth of the Bosphorus Strait on the Asian mainland, lying to the east of the city of Istanbul that lies on the Turquoise Coast.\nHuman: Can you speak Turkish?\nAI: No\nHuman: Türkçe konuşabilir misin?\nAI: It's wrong.\nHuman: the next input\nAI:"

In [71]:
def chat_to_llm(chat_llm ):
    conversation_total_tokens = 0
    new_conversation = ConversationChain(llm=chat_llm, 
                                     verbose=False, 
                                     memory=ConversationBufferMemory()
                                     )
    
    while True:
        message = input("Human: ")
        if message=='exit':
            print(f"{conversation_total_tokens} tokens used in total in this conversation")
            break
        if message:
            formatted_prompt = conversation.prompt.format(input=message,history=new_conversation.memory.buffer)
            num_tokens = len(tokenizer.tokenize(formatted_prompt))
            conversation_total_tokens += num_tokens
            print(f'tokens sent {num_tokens}')
            response = new_conversation.predict(input=message)
            response_num_tokens = len(tokenizer.tokenize(response))
            conversation_total_tokens += response_num_tokens
            print(f"LLM: {response}")


    

In [72]:
chat_to_llm(flan_ul2)

Human: How are you today?
tokens sent 64


KeyboardInterrupt: ignored

In [ ]:
chat_to_llm(flan_t5)


Human: hi how are you?
tokens sent 64
LLM: I am good thanks.
Human: What is your name?
tokens sent 78
LLM: I am a robot.
Human: exit
153 tokens used in total in this conversation


## Using a Summary Memory

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

In [ ]:
summary_memory = ConversationSummaryMemory(llm=flan_ul2)

conversation = ConversationChain(
    llm=flan_ul2, 
    verbose=True, 
    memory=summary_memory
)

In [ ]:
conversation.predict(input="Hi there! I am Sam")

In [ ]:
conversation.predict(input="How are you today?")

In [ ]:
conversation.predict(input="I am good. What is your name?")

In [ ]:
conversation.predict(input="Thats cool can you help me with customer support?")

In [ ]:
conversation.predict(input="What time is your shop open today?")